In [1]:
# Import dependencies
import os
from pathlib import Path
import numpy as np
import pandas as pd

from pyAudioAnalysis import audioBasicIO, ShortTermFeatures

import matplotlib.pyplot as plt
import librosa
import librosa.display
from tqdm import tqdm
import tensorflow as tf
from keras.preprocessing.image import img_to_array
from PIL import Image as pil_Image
from IPython.display import display, Image

In [2]:
# Retrieve the audio files
audio_path = "../resources/audio_files/"
audio_files = os.listdir(audio_path)

# Ignore duplicates
duplicates = ['voice005.wav', 'voice006.wav', 'voice054.wav', 'voice055.wav']

In [8]:
import pylab

# Create the spectrograms
for file in tqdm(audio_files, desc="Creating spectrograms"):
    
    # Only read .wav files
    if (file.endswith(".wav")) and (file not in duplicates):
        
        # Load the file
        y, sr = librosa.load(
            audio_path + file, # full file path
            sr = None # preserve sampling rate
        )
        
        # Use pylab
        pylab.specgram(y, Fs=sr)
        

        # # Plot the spectrogram
        # D = librosa.amplitude_to_db(
        #     np.abs(librosa.stft(y)),
        #     ref = np.max
        # )

        # Plot the spectrogram
        # librosa.display.specshow(
        #     D,
        #     sr = sr,
        #     x_axis = 'time',
        #     y_axis = 'linear' # can also choose: linear
        # )
        
        # Define the filename
        filename = file.split(".")[0]

        # Remove labels and border
        plt.tight_layout()
        plt.axis('off')
        # plt.show()
        # Export image
        plt.savefig(
            f'../resources/spectrograms/pylab/{filename}.png',
            bbox_inches = 'tight',
            pad_inches = 0
        )
        
        # Close the figure to avoid runtime warning
        plt.close()

Creating spectrograms: 100%|██████████████████| 209/209 [00:23<00:00,  8.93it/s]


In [9]:
# Define the image path and files
image_path = "../resources/spectrograms/pylab/"
image_files = os.listdir(image_path)

In [17]:
# Resize each image
# new_width, new_height = 305, 225
new_width, new_height = 322, 322

for image_name in tqdm(image_files, desc="Resizing spectrograms"):
    
    # Only read .png files
    if image_name.endswith(".png"):
        
        # Open the image file
        img = pil_Image.open(image_path + image_name)
        
        # Resize
        resized = img.resize((new_width, new_height))
        
        # Create a new figure
        plt.figure(figsize=(new_width / 100, new_height / 100))
        # plt.figure(figsize=(new_width, new_height))
        
        # Plot the resized image
        plt.imshow(resized)

        # Define the filename
        filename = image_name.split(".")[0]
        
        # Remove labels and border
        plt.tight_layout()
        plt.axis('off')
        
        # Export image
        plt.savefig(
            f'../resources/spectrograms/pylab_resize/{filename}.png',
            bbox_inches = 'tight',
            pad_inches = 0
        )
        
        # Close the figure to avoid runtime warning
        plt.close()

Resizing spectrograms: 100%|██████████████████| 204/204 [00:13<00:00, 14.83it/s]


In [18]:
# Define the resized image path and files
# resized_path = "../resources/spectrograms/resized/"
resized_path = "../resources/spectrograms/pylab_resize/"
resized_files = os.listdir(resized_path)

In [19]:
# Initialise a list to hold the dictionaries
spectro_list = []
id_list = []
r_list = []
g_list = []
b_list = []
a_list = []

# Loop through each image
for resized_image in resized_files:
    
    # Only read .png files
    if resized_image.endswith(".png"):
        
        # Initialise a dictionary to hold the pixels
        spectro_dict = dict()

        # Open the image file
        img = pil_Image.open(resized_path + resized_image)

        # Convert image to array format
        img_array = img_to_array(img)
        
        # Add image attributes and array to dictionary
        spectro_dict['id'] = resized_image.split(".")[0]
        spectro_dict['format'] = img.format
        spectro_dict['mode'] = img.mode
        spectro_dict['width_px'] = img.width
        spectro_dict['height_px'] = img.height
        
        id_list.append(resized_image.split(".")[0])
        r_list.append(img_array[:, :, 0].flatten().astype(int))
        g_list.append(img_array[:, :, 1].flatten().astype(int))
        b_list.append(img_array[:, :, 2].flatten().astype(int))
        a_list.append(img_array[:, :, 3].flatten().astype(int))
        
        spectro_list.append(spectro_dict)

# Create a list of RGBA lists
rgba_list = [r_list, g_list, b_list, a_list]

# Convert the list to a DataFrame
spectro_df = pd.DataFrame(spectro_list)
spectro_df.head()

,id,format,mode,width_px,height_px
0,voice156,PNG,RGBA,256,256
1,voice142,PNG,RGBA,256,256
2,voice195,PNG,RGBA,256,256
3,voice181,PNG,RGBA,256,256
4,voice022,PNG,RGBA,256,256


In [20]:
# Colour reference list
colours = ['r', 'g', 'b', 'a']

# Loop through each file
for idx, colour_list in tqdm(enumerate(rgba_list), desc="Exporting as CSV"):
    
    # Create a dataframe of each colour
    df = pd.DataFrame(colour_list)
    
    # Use id as the index to the dataframe
    df.index = id_list
    
    # Export to CSV
    df.transpose().to_csv(
        f'../resources/clean_data/spectrogram/pylab{colours[idx]}val.csv',
        encoding = 'utf8',
        index = False
    )

Exporting as CSV: 4it [01:07, 16.98s/it]
